In [1]:
# Entree 1

import numpy as np
import cv2
import pandas as pd
import os
import json

path = "C:/Users/adeik/MaunaKea/Train/"
patients = {}

# Creer un dict avec tous les paths des images par patient
for i in range(61):
    patients[str(i)] = []
    for file in os.listdir(path):
        if file.endswith("_" + str(i) + ".png"):
            patients[str(i)] += [file]

# creer un json pour recupérer le dict apres
with open('unsorted_patients.txt', 'w') as json_file:
    json.dump(patients, json_file)

targets = pd.read_csv("train_target.csv")
targets['patient'] = targets['image_filename'].map(lambda x: x.partition('_')[-1].partition('_')[-1].partition('.')[0])
targets = targets.sort_values(by = ['patient', 'image_filename']).set_index(keys = ['image_filename'])


In [2]:
# lancer plusieurs fois pour virer les clés vides

keys = list(patients.keys())
for key in keys:
    if (patients[key] == []):
        patients.pop(key)
        
# Entree 19

In [3]:
targets.to_csv('new_train_targets.csv', sep = ',')


In [22]:
#help(targets)


In [6]:
# Pour que les images soient dans "l'ordre"
for key in list(patients.keys()):
    for i in range(len(patients[key])):
        if (int(patients[key][i].partition('_')[-1].partition('_')[0]) < 10):
            patients[key][i] = patients[key][i].partition('_')[0] + '_0' + patients[key][i].partition('_')[-1]
    patients[key].sort()

# Widget d'exploration des images

nécéssite ipywidgets et bqplot

In [7]:
import ipywidgets as widgets
from IPython.display import display
from bqplot import OrdinalScale, LinearScale, Bars, Lines, Axis, Figure

patient_keys = []

for key in patients.keys():
    patient_keys += [key]
    
# Widgets
if(True):
    patient_number = widgets.Dropdown(options = patient_keys,
                                      value = '0',
                                      description = 'Patient:',
                                      disabled = False)

    file = open(path + patients[patient_number.value][-1], 'rb')
    image = file.read() 

    patient_image = widgets.Image(value = image,
                                  format = 'png',
                                  width = 300,
                                  height = 300)

    patient_class = widgets.Text(value = '0 : Squamous Epithelium',
                                 disabled = True)
    
    patient_play = widgets.Play(value = 0,
                                min = 0,
                                max = len(patients[patient_number.value]) - 1,
                                interval = 150,
                                step = 1,
                                disabled = False)

    patient_next = widgets.Button(description = 'Next',
                                  disabled = False)

    patient_prev = widgets.Button(description = 'Prev',
                                  disabled = False)

    patient_buttons = widgets.HBox([patient_prev, patient_next])
    
# Freq plot
if(True):
    size = 4

    x_data = np.arange(4)

    x_ord = OrdinalScale(domain = [0, 1, 2, 3])
    y_sc = LinearScale(min = 0, max = 1)

    bar = Bars(x = x_data,
               y = np.zeros(size),
               scales={'x': x_ord, 'y': y_sc},
               type='stacked')

    ax_x = Axis(scale=x_ord,
                grid_lines='solid', label='Class Number')

    ax_y = Axis(scale=y_sc,
                orientation='vertical',
                tick_format='0.2f',
                grid_lines='solid',
                label='Frequency')

    freq_plot = Figure(marks = [bar],
                       axes = [ax_x, ax_y],
                       title='Patient Class Distribution',
                       animation_duration = 1000)

    new_y = np.zeros(4)
    new_y[targets[targets['patient'] == '0']['class_number'].value_counts().index] = targets[targets['patient'] == '0']['class_number'].value_counts()
    new_y/= new_y.sum()
    bar.y = new_y

# Callbacks
if(True):
    def next_patient(change):
        if (int(patient_number.value) < int(patient_keys[-1])):
            patient_number.value = patient_number.options[patient_number.options.index(patient_number.value) + 1]
            new_y = np.zeros(4)
            new_y[targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts().index] = targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts()
            new_y/= new_y.sum()
            bar.y = new_y
            patient_prev.disabled = False
        else:
            patient_next.disabled = True
    
    def select_patient(change):
            new_y = np.zeros(4)
            new_y[targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts().index] = targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts()
            new_y/= new_y.sum()
            bar.y = new_y

    def prev_patient(change):
        if (int(patient_number.value) > int(patient_keys[0])):
            patient_number.value = patient_number.options[patient_number.options.index(patient_number.value) - 1]
            new_y = np.zeros(4)
            new_y[targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts().index] = targets[targets['patient'] == str(patient_number.value)]['class_number'].value_counts()
            new_y/= new_y.sum()
            bar.y = new_y
            patient_next.disabled = False
        else:
            patient_prev.disabled = True

    def patient_playspeed(change):
        if(patient_speed == 'Slow'):
            patient_play.interval = 500

        if(patient_speed == 'Regular'):
            patient_play.interval = 250

        if(patient_speed == 'Fast'):
            patient_play.interval = 100

    def patient_animate(change):
        patient_play.max = len(patients[patient_number.value]) - 1
        
        # image update
        if(True):
            if (patient_play.value < 10):
                image_path = patients[patient_number.value][patient_play.value].partition('_')[0] + '_' + \
                             str(int(patients[patient_number.value][patient_play.value].partition('_')[-1].partition('_')[0])) + \
                             '_' + patients[patient_number.value][patient_play.value].partition('_')[-1].partition('_')[-1]
            else:
                    image_path = patients[patient_number.value][patient_play.value]
            file = open(path + image_path, 'rb')
            image = file.read() 
            patient_image.value = image
        
        # class update
        if(True):
            if(targets.loc[image_path].class_number == 0):
                patient_class.value = '0 : Squamous Epithelium'

            if(targets.loc[image_path].class_number == 1):
                patient_class.value = '1 : Intestinal Metaplasia'

            if(targets.loc[image_path].class_number == 2):
                patient_class.value = '2 : Gastric Metaplasia'

            if(targets.loc[image_path].class_number == 3):
                patient_class.value = '3 : Dysplasia and Cancer'

# Observes
if(True):
    patient_play.observe(patient_animate, 'value')
    patient_next.on_click(next_patient)
    patient_prev.on_click(prev_patient)
    patient_number.observe(select_patient)

# Boxes
if(True):
    patient_box = widgets.VBox([patient_image, patient_number,patient_class, patient_play, patient_buttons])
    widget_box = widgets.HBox([patient_box, freq_plot])
    display(widget_box)

- Patients 11, 12, 32, 51, 53 sont multi classe
- Essayer mutli resolution processing
- Essayer de split les images en a peu près 28x28 pour avoir un problème similaire au MNIST?
- Essayer un modele shared vision pour savoir si on doit classifier le patient partout pareil ou si on doit classifier image par image?



> * 1 : Entrainer un cnn avec mutli resolution processing sur single frame
>    * 1.1 : faire la classification de l'image entière par aggregation des classifications
>
> Si on classifie les patients de manière unique :
> * 2 : prendre les outputs des probas et les entraîner sur dense en prenant une taille d'input fixe de 27
> * 3 : prendre les outputs des probas et les entrainer sur RNN avec input variable pour évaluer sur toutes les photos
>
> Si on classifie les patients image par image :
> * 4 : trouver une façon de détecter des images 'outliers'

Video Classification:

https://medium.com/@gcagrici/video-classification-with-deep-learning-3840b20b7949

https://towardsdatascience.com/introduction-to-video-classification-6c6acbc57356

Keras Functional API / Shared Vision: 

https://keras.io/getting-started/functional-api-guide/